- link: 
    - [subprocess — Spawning Additional Processes](https://pymotw.com/3/subprocess/index.html#82b562ef-2cfa-a49e-4482-5c5d17a067d1)

- Three APIs for working with processes.
    - ```run()```
    - ```call()```, ```check_all()```, ```check_output()```
    - ```Popen```

- The *subprocess* module is intended to **replace** functions 
    - such as ```os.system()```, ```os.spawnv()```, 
    - and the variations of ```popen()``` in the *os* and *popen2* modules.

- Also note that:
    - Some file can only get the **right** result ***outside the Jupyter Notebook***
    - i.e. inside a ```.py``` file 

In [4]:
import subprocess

### Running External Command

- right on!

In [5]:
# passin' list 
completed = subprocess.run(['pwd','-P'])

completed.args
completed.returncode  # 0: success 

['pwd', '-P']

0

- *subprocess* in the shell XD

In [6]:
completed = subprocess.run(
    'echo $HOME', 
    shell=True, 
    stdout=subprocess.PIPE  # optional if not in Jupyter Notebook
)

print(completed.stdout.decode('utf-8'))

/Users/alex



- ```check``` with error handling

In [7]:
# fine without checking
subprocess.run(["false"])


try:
    
    # checking code - not zero? => error was raised 
    subprocess.run(["false"], check=True)
    
except subprocess.CalledProcessError as err:
    print(err)

CompletedProcess(args=['false'], returncode=1)

Command '['false']' returned non-zero exit status 1.


- *throw* output into the ```PIPE```

In [9]:
completed = subprocess.run(
    ['ls', '-t', '../..'],
    stdout=subprocess.PIPE  # PIPE <=> xx.stdout.decode(...)
)

print(
    'return-code: {}\n{} bytes in stdout'.format(
        completed.returncode,
        len(completed.stdout)
    )
)

print(
    '\n{}'.format(
        completed.stdout.decode('utf-8')  # byte.stdout -> str
    )
)

return-code: 0
345 bytes in stdout

README.md
file_and_directory_access
python_language_services
interprocess_commu_and_networkin
concurrent_execution
__HOWTOs__
builtin_stuff
internet_protocols_and_support
numeric_and_mathematical_modules
text_processing_services
cryptographic_services
data_types
functional_programming_modules
internet_data_handling
generic_os_services
LICENSE



- ```check_output()``` with ```check```, ```stdout```

In [13]:
completed = subprocess.run(
    ['ls', '-t', '../..'],
    check=True,              # the only difference with previous one 
    stdout=subprocess.PIPE   # getting val by 'xx.stdout.decode(xx)'
)
 
# Or using 'check_output()'  # same functionality

completed_v2 = subprocess.check_output(
    ['ls', '-t', '../..']       # getting val by 'xx.decode(xx)' 
)


# print(completed.stdout.decode('utf-8'))
print(completed_v2.decode('utf-8'))

README.md
file_and_directory_access
python_language_services
interprocess_commu_and_networkin
concurrent_execution
__HOWTOs__
builtin_stuff
internet_protocols_and_support
numeric_and_mathematical_modules
text_processing_services
cryptographic_services
data_types
functional_programming_modules
internet_data_handling
generic_os_services
LICENSE



- output error (to the console)
    > I **DO NOT** fully understand this little chapter.

In [14]:
try:
    completed = subprocess.run(
        'exit 1',
        check=True,
        shell=True,
        stdout=subprocess.PIPE  
    )
    
except subprocess.CalledProcessError as err:
    print(err)  # the error was printed to console 
    
else:
    
    # the rest of the result was hidden (to console)
    
    print(
        'return-code: {}\n{} bytes in stdout'.format(
            completed.returncode,
            len(completed.stdout)
        )
    )

    print(
        '\n{}'.format(
            completed.stdout.decode('utf-8')  
        )
    )

Command 'exit 1' returned non-zero exit status 1.


In [15]:
try:
    completed = subprocess.run(
        
        # "1>&2" means redirect stdout to stderr
        'echo to stdout; echo to stderr 1>&2; exit 1',
        
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    
except subprocess.CalledProcessError as err:
    print(err)
    
else:
    
    print('return-code: {}'.format(completed.returncode))
    
    print('{} bytes in stdout: {!r}'.format(
        len(completed.stdout),
        completed.stdout.decode('utf-8')
    ))
    
    print('{} bytes in stderr: {!r}'.format(
        len(completed.stdout),
        completed.stdout.decode('utf-8')
    ))

return-code: 1
10 bytes in stdout: 'to stdout\n'
10 bytes in stderr: 'to stdout\n'


- capture err msg when using ```check_output()```

In [16]:
try:
    output = subprocess.check_output(
        'echo to stdout; echo to stderr 1>&2',
        shell=True,
        stderr=subprocess.STDOUT,
    )
    
except subprocess.CalledProcessError as err:
    print('Error:', err)
    
else:
    
    print('{} bytes in output: {!r}'.format(
        len(output),
        output.decode('utf-8')
    ))

20 bytes in output: 'to stdout\nto stderr\n'


- suppressing output

In [17]:
# This case is for 
#   "the output should not be shown or captured" sort of thing.

try:
    completed = subprocess.run(
        'echo to stdout; echo to stderr 1>&2; exit 1',
        shell=True,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    
except subprocess.CalledProcessError as err:
    print('Error:', err)
    
else:
    
    print('return-code:', completed.returncode)
    
    print('stdout is {!r}'.format(completed.stdout))
    print('stderr is {!r}'.format(completed.stderr))

return-code: 1
stdout is None
stderr is None


### Working with Pipes directly 

In [18]:
import subprocess

- One-way commu with a process

In [33]:
"""[Read]"""

proc = subprocess.Popen(
    ['echo', '"to stdout"'],
    stdout=subprocess.PIPE,
)

stdout_val = proc.communicate()[0].decode('utf-8')
print('stdout: ', repr(stdout_val))

'[Read]'

stdout:  '"to stdout"\n'


<hr>

```python
"""[Write]"""

import subprocess 

proc = subprocess.Popen(
    ['cat', '-'],
    stdin=subprocess.PIPE,
)

proc.communicate('stdin: to stdin\n'.encode('utf-8'))
```

<hr>

In [19]:
!python3 ./subprocess_pyfile_only/01-workin-pipe-1way-write.py 

stdin: to stdin


- Bi-directional commu with a process

In [20]:
"""[Popen2]"""

proc = subprocess.Popen(
    ['cat', '-'],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
)

msg        = 'through stdin to stdout'.encode('utf-8')
stdout_val = proc.communicate(msg)[0].decode('utf-8')

print('pass through: ', repr(stdout_val))

'[Popen2]'

pass through:  'through stdin to stdout'


- Capturing error output

In [21]:
"""[Popen3]"""

proc = subprocess.Popen(
    ['cat -; echo "to stderr" 1>&2'],
    shell=True,
    stdin =subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

msg = 'through stdin to stdout'.encode('utf-8')

stdout_val, stderr_val = proc.communicate(msg)

print(
    'pass through: ', repr(stdout_val.decode('utf-8')))
print(
    'stderr      : ', repr(stderr_val.decode('utf-8')))

'[Popen3]'

pass through:  'through stdin to stdout'
stderr      :  'to stderr\n'


- Combining regular & error output

In [24]:
"""[Popen4]"""

proc = subprocess.Popen(
    'cat -; echo "to stderr" 1>&2',
    shell=True,
    stdin =subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,                   # MARK
)

msg = 'through stdin to stdout\n'.encode('utf-8')

stdout_val, stderr_val = proc.communicate(msg)  # MARK

print(
    'combined output: ', repr(stdout_val.decode('utf-8')))
print(
    'stderr val     : ', repr(stderr_val))

'[Popen4]'

combined output:  'through stdin to stdout\nto stderr\n'
stderr val     :  None


### Connecting Segments of a Pipe

In [25]:
import subprocess

In [28]:
ls = subprocess.Popen(
    ['ls', '../..'],
    stdout=subprocess.PIPE,  # MARK
)

grep = subprocess.Popen(
    ['grep', 'services'],
    stdin=ls.stdout,         # MARK
    stdout=subprocess.PIPE,  
)

end_of_pipe = grep.stdout

for line in end_of_pipe:
    print(line.decode('utf-8').strip())

cryptographic_services
generic_os_services
python_language_services
text_processing_services


### Interacting with Another Command

In [34]:
# Examples are inside the .py file.
#   Instructions 
#     run 'python3 *master*'  (aha)

!ls ./subprocess_pyfile_only/*intract_othercmd*

./subprocess_pyfile_only/03-intract_othercmd_slave_repeater.py
./subprocess_pyfile_only/04-intract_othercmd_master_interact.py


### Signaling Between Processes